In [102]:
%run Model_2.ipynb

In [103]:
import pytest
import numpy as np
from sklearn.naive_bayes import GaussianNB as SklearnGaussianNB

np.random.seed(0)


# Check 1: `_gaussian_log_pdf`

**Test 1.1 : Gaussian log pdf at the mean**

checks that our implementation of `_gaussian_log_pdf` matches the closed-form analytical value at the mean.  

For $N(0, 1)$, log pdf at $x=0$ is $-0.5 * \log(2*pi)$



In [104]:
m = GaussianNaiveBayes()

val = m._gaussian_log_pdf(0.0, mean=0.0, var=1.0)

expected = -0.5 * np.log(2.0 * np.pi * 1.0)

print("Computed:", val)
print("Expected:", expected)
assert val == pytest.approx(expected, rel=1e-6)
print("Test 1.1 PASSED: gaussian_log_pdf center value correct.")



Computed: -0.9189385332046727
Expected: -0.9189385332046727
Test 1.1 PASSED: gaussian_log_pdf center value correct.


**Test 1.2: Monotonic decay away from the mean**

checks that our implementation of `_gaussian_log_pdf` gives correct Gaussian log density result that will monotonic decay as we move away from the mean.

For a fixed normal distribution $N(0, 1)$, the log pdf should be highest at the mean and decrease as $|x - \mu|$ increases.


In [105]:
center = m._gaussian_log_pdf(0.0, mean=0.0, var=1.0)
off_1  = m._gaussian_log_pdf(1.0, mean=0.0, var=1.0)
off_2  = m._gaussian_log_pdf(2.0, mean=0.0, var=1.0)
print(center, off_1, off_2)

assert center > off_1 > off_2
print("Test 1.2 PASSED: gaussian_log_pdf monotonicity correct.")


-0.9189385332046727 -1.4189385332046727 -2.9189385332046727
Test 1.2 PASSED: gaussian_log_pdf monotonicity correct.


**Test 1.3: Effect of the variance on the log pdf**

checks that changing the variance parameter in the Gaussian distribution changes the output of `_gaussian_log_pdf`.



In [106]:
n1 = m._gaussian_log_pdf(1.0, mean=0.0, var=0.5)
n2 = m._gaussian_log_pdf(1.0, mean=0.0, var=5.0)

assert not np.isclose(n1, n2)
print("Test 1.3 PASSED: variance affects gaussian_log_pdf output.")


Test 1.3 PASSED: variance affects gaussian_log_pdf output.


# Check 2: `_compute_log_likelihood`

**Test 2.1: calculation of log likelihood value**

checks if `_compute_log_likelihood` correctly sums the per-feature Gaussian log pdfs by manually calculating total log-likelihood from a 2-dimensional toy dataset example.

In [107]:
m.classes_ = np.array([0])
m.gaussian_means_[0] = {0: 0.0, 1: 1.0}
m.gaussian_vars_[0]  = {0: 1.0, 1: 4.0}

x_row = np.array([0.0, 3.0])
log1 = m._gaussian_log_pdf(x_row[0], 0.0, 1.0)
log2 = m._gaussian_log_pdf(x_row[1], 1.0, 4.0)
expected = log1 + log2

assert m._compute_log_likelihood(x_row, 0) == pytest.approx(expected, rel=1e-6)
print("Test 2.1 PASSED: compute_log_likelihood matches manual calculation.")


Test 2.1 PASSED: compute_log_likelihood matches manual calculation.


**Test 2.2： distance and log-likelihood**

checks that samples farther from the class mean produce lower log-likelihood values. 

In [108]:
x1 = np.array([0.0, 1.0])
x2 = np.array([5.0, 10.0])

ll1 = m._compute_log_likelihood(x1, 0)
ll2 = m._compute_log_likelihood(x2, 0)

assert ll1 > ll2
print("Test 2.2 PASSED: log-likelihood decreases for far-away samples.")


Test 2.2 PASSED: log-likelihood decreases for far-away samples.


# Check 3: `fit`

**Test 3.1: Correct computation of class priors and class structure**

checks that `fit()` correctly detects class labels and calculates the corresponding log-priors based on class frequencies. 


In [109]:
X_toy = np.array([[1,2],[1.2,1.9],[3.2,4.8],[3,5.1],[2.5,2.9]])
y_toy = np.array([0,0,1,1,1])

m.fit(X_toy, y_toy)

# classes
assert isinstance(m.classes_, np.ndarray)
assert m.classes_.ndim == 1
assert set(m.classes_) == {0,1}

assert set(m.log_priors_.keys()) == set(m.classes_)


N = len(y_toy)
for c in m.classes_:
    expected = np.log(np.sum(y_toy == c) / N)
    assert np.isclose(m.log_priors_[c], expected)

print("Test 3.1 PASSED: fit() correctly computes priors and classes.")


Test 3.1 PASSED: fit() correctly computes priors and classes.


**Test 3.2: Positive variances after smoothing**

checks that all per-feature variances are strictly positive after smoothing. 


In [110]:

for c in m.classes_:
    for j in range(X_toy.shape[1]):
        assert m.gaussian_vars_[c][j] > 0

print("Test 3.2 PASSED: fit() produces positive variances.")


Test 3.2 PASSED: fit() produces positive variances.


**Test 3.3: Correct structure of stored means and variances**

checks that each class stores a full set of per-feature means and variances with the correct dictionary structure.

In [111]:

n_features = X_toy.shape[1]

for c in m.classes_:
    assert len(m.gaussian_means_[c]) == n_features
    assert len(m.gaussian_vars_[c]) == n_features

print("Test 3.3 PASSED: fit() structure for means/vars correct.")


Test 3.3 PASSED: fit() structure for means/vars correct.


**Test 3.4: NaN in input should raise ValueError**

when there's NaN value in fit() input, raise ValueError. This ValueError is included in the model since GNB could not process NaN values.


In [112]:
try:
    X_nan = np.array([[1.0, 2.0],
                      [np.nan, 3.0]])
    y_nan = np.array([0, 1])

    m.fit(X_nan, y_nan)
    raise AssertionError("Test 3.4 FAILED: fit did not raise ValueError on NaN input.")
except ValueError:
    print("Test 3.4 PASSED: fit raises ValueError when X contains NaN.")


Test 3.4 PASSED: fit raises ValueError when X contains NaN.


# Check 4: `predict_log_proba`

**Test 4.1: Output shape and dtype of `predict_log_proba`**

checks that `predict_log_proba` method returns the output shape (n_sample, n_classes) and dtype.  


In [113]:

X_test = np.array([[1.1,2.0],[3.1,4.9]])

logp = m.predict_log_proba(X_test)
assert isinstance(logp, np.ndarray)
assert logp.shape == (2,2)
assert logp.dtype == float

print("Test 4.1 PASSED: predict_log_proba shape + dtype correct.")


Test 4.1 PASSED: predict_log_proba shape + dtype correct.


**Test 4.2: `predict_log_proba` scores**

checks that the more distant the samples, the lower the log-posterior scores.


In [114]:
lp1 = m.predict_log_proba([[1.1,2.0]])[0]
lp2 = m.predict_log_proba([[100,100]])[0]

assert lp1.max() > lp2.max()
print("Test 4.2 PASSED: predict_log_proba scores changes correspond to the distance.")


Test 4.2 PASSED: predict_log_proba scores changes correspond to the distance.


# Check 5: `predict_proba`

**Test 5.1: Probability normalization**

check that each probability row from `predict_proba` sums to 1.


In [115]:

probs = m.predict_proba(X_test)
assert np.allclose(probs.sum(axis=1), 1.0)

print("Test 5.1 PASSED: predict_proba rows sum to 1.")


Test 5.1 PASSED: predict_proba rows sum to 1.


**Test 5.2: Non negative probability**

checks all values returned by `predict_proba` are non-negative.


In [116]:
assert np.all(probs >= 0)
print("Test 5.2 PASSED: predict_proba produces non-negative probability.")


Test 5.2 PASSED: predict_proba produces non-negative probability.


**Test 5.3: Consistency between log-probability and probability**

checks that the class with the highest log-probability also has the highest probability after normalization.

In [117]:
# class with max log-prob = class with max prob
logp = m.predict_log_proba(X_test)
probs = m.predict_proba(X_test)

assert np.argmax(logp[0]) == np.argmax(probs[0])
print("Test 5.3 PASSED: predict_proba aligns with predict_log_proba.")


Test 5.3 PASSED: predict_proba aligns with predict_log_proba.


# Check 6: `predict`

**Test 6.1: Output shape and valid class labels**

checks that `predict` returns a 1D array of valid class labels.


In [118]:
preds = m.predict(X_test)

assert preds.shape == (X_test.shape[0],)
assert set(preds).issubset(set(y_toy))

print("Test 6.1 PASSED: predict shape and value set correct.")

Test 6.1 PASSED: predict shape and value set correct.


**Test 6.2: Consistency with `predict_proba`**

checks that `predict` always chooses the class with the highest predicted probability.


In [119]:
probs = m.predict_proba(X_test)
assert np.array_equal(preds, np.argmax(probs, axis=1))

print("Test 6.2 PASSED: predict matches argmax of probability.")


Test 6.2 PASSED: predict matches argmax of probability.


# Check 7: Edge cases

**Test 7.1: zero-variance feature**

checks the case where a feature has variance 0 within each class. Confirms variance smoothing prevents divide-by-zero and that predict_log_proba()/predict_proba() remain finite and normalized. 

In [120]:

X_zero_var = np.array([
    [1.0, 2.0],
    [1.0, 3.0],
    [1.0, 4.0],
    [1.0, 5.0],
])
y_zero_var = np.array([0, 0, 1, 1])

m.fit(X_zero_var, y_zero_var)

logp_zero = m.predict_log_proba(X_zero_var)
probs_zero = m.predict_proba(X_zero_var)

assert np.isfinite(logp_zero).all(), "Test 7.1 FAILED: predict_log_proba produced non-finite values with zero-variance feature."
assert np.isfinite(probs_zero).all(), "Test 7.1 FAILED: predict_proba produced non-finite values with zero-variance feature."
assert np.allclose(probs_zero.sum(axis=1), 1.0, atol=1e-6), "Test 7.1 FAILED: probabilities do not sum to ~1."

print("Test 7.1 PASSED: zero-variance features are handled correctly via variance smoothing.")



Test 7.1 PASSED: zero-variance features are handled correctly via variance smoothing.


**Test 7.2: Single-class training**

checks that training on data with only one class does not crash and produces reasonable outputs. Ensures classes_ has length 1, predict_proba() returns probabilities ≈ 1, and predict() always returns the only class.

In [121]:
X_one_class = np.array([
    [1.0, 2.0],
    [1.5, 2.5],
    [2.0, 3.0],
])
y_one_class = np.array([0, 0, 0])

m.fit(X_one_class, y_one_class)

assert m.classes_.shape == (1,), "Test 7.2 FAILED: classes_ should contain exactly one class."
assert m.classes_[0] == 0, "Test 7.2 FAILED: the single class in classes_ should be 0."

probs_one_class = m.predict_proba(X_one_class)
preds_one_class = m.predict(X_one_class)

assert probs_one_class.shape == (X_one_class.shape[0], 1), "Test 7.2 FAILED: predict_proba shape incorrect for single-class training."
assert np.allclose(probs_one_class, 1.0, atol=1e-6), "Test 7.2 FAILED: probabilities are not all ~1 for single-class training."
assert np.array_equal(preds_one_class, np.zeros_like(y_one_class)), "Test 7.2 FAILED: predict did not return the only class."

print("Test 7.2 PASSED: model behaves correctly when trained on a single class.")

Test 7.2 PASSED: model behaves correctly when trained on a single class.


**Test 7.3: Single test sample**

verifies shape handling and probability normalization when predicting on a single input sample. Ensures predict_proba() returns shape (1, C), sums to 1, and predict() returns a valid class label.

In [122]:
X_train_small = np.array([
    [2.0, 3.0],
    [3.0, 4.0],
    [4.0, 5.0],
])
y_train_small = np.array([0, 1, 1])
X_test_single = np.array([[3.5, 4.5]])

m.fit(X_train_small, y_train_small)

probs_test_single = m.predict_proba(X_test_single)
pred_test_single = m.predict(X_test_single)

assert probs_test_single.shape == (1, 2), "Test 7.3 FAILED: predict_proba shape incorrect for a single test sample."
assert np.allclose(probs_test_single.sum(axis=1), 1.0, atol=1e-6), "Test 7.3 FAILED: probs do not sum to ~1."
assert pred_test_single.shape == (1,), "Test 7.3 FAILED: predict shape incorrect for a single test sample."
assert pred_test_single[0] in m.classes_, "Test 7.3 FAILED: predicted label not in known classes."

print("Test 7.3 PASSED: predict_proba/predict handle a single test sample correctly.")


Test 7.3 PASSED: predict_proba/predict handle a single test sample correctly.


**Test 7.4: Extreme magnitudes**

tests numerical stability under very large feature values. Confirms log-sum-exp and log-space computations avoid underflow/overflow and keep outputs finite and normalized.

Large values stress (x-mean)^2 / var. We only require finite outputs and normalized probabilities.

In [123]:

X_extreme = np.array([
    [ 1e9,  -1e9],
    [ 1e9+1, -1e9-1],
    [-1e9,   1e9],
    [-1e9-1, 1e9+1],
])
y_extreme = np.array([0, 0, 1, 1])

m.fit(X_extreme, y_extreme)

logp_ext = m.predict_log_proba(X_extreme)
probs_ext = m.predict_proba(X_extreme)

assert np.isfinite(logp_ext).all(), "Test 7.4 FAILED: log-probabilities contain NaN/inf under extreme magnitudes."
assert np.isfinite(probs_ext).all(), "Test 7.4 FAILED: probabilities contain NaN/inf under extreme magnitudes."
assert np.allclose(probs_ext.sum(axis=1), 1.0, atol=1e-6), "Test 7.4 FAILED: probs do not sum to ~1 under extreme magnitudes."

print("Test 7.4 PASSED: extreme magnitudes remain numerically stable (log-space + log-sum-exp).")


Test 7.4 PASSED: extreme magnitudes remain numerically stable (log-space + log-sum-exp).


Overall, these tests demonstrates that each component of our Gaussian Naive Bayes implementation works correctly in isolation, handles edge cases correctly, and exactly reproduces both the predictions and probabilistic outputs of sklearn’s implementation on a real-world dataset. 